# THIS FILE WAS USED TO CREATE TABLEAU READY CSV FILES AS A WORKAROUND TO CONNECTING PGADMIN TO TABLEAU

however, we found the solution which was *actually* follow the install guide of the driver. Note it is necesarry to show "hidden" files in the username folder

In [1]:
import pandas as pd

In [2]:
# Read cleaned_calendar.csv
calendar_df = pd.read_csv('cleaned_data/cleaned_calendar.csv')

# Read cleaned_listings.csv
listings_df = pd.read_csv('cleaned_data/cleaned_listings.csv')

# Read cleaned_reviews.csv
reviews_df = pd.read_csv('cleaned_data/cleaned_reviews.csv')

### Dimension Neighborhood

In [8]:
# Create Dimension Neighborhood with Auto-Incrementing ID
dimension_neighborhood_df = listings_df[['neighbourhood_cleansed']].drop_duplicates()
dimension_neighborhood_df.rename(columns={
    'neighbourhood_cleansed': 'neighborhood'
}, inplace=True)

# Add an auto-incrementing ID column
dimension_neighborhood_df.reset_index(drop=True, inplace=True)
dimension_neighborhood_df.insert(0, 'neighborhood_id', range(1, len(dimension_neighborhood_df) + 1))

# Save to CSV
#dimension_neighborhood_df.to_csv('analytical_database/dimension_neighborhood.csv', index=False)

In [9]:
dimension_neighborhood_df.head()

,neighborhood_id,neighborhood
0,1,Vesterbro-Kongens Enghave
1,2,Nørrebro
2,3,Amager Øst
3,4,Østerbro
4,5,Indre By


### Dimension Listing

In [4]:
# Create Dimension Listing
dimension_listing_df = listings_df[[
    'id', 'listing_url', 'picture_url', 'name', 'description',
    'neighborhood_overview', 'latitude', 'longitude', 'property_type',
    'room_type', 'amenities', 'minimum_nights', 'maximum_nights', 'instant_bookable'
]].copy()

dimension_listing_df.rename(columns={'id': 'listing_id'}, inplace=True)
dimension_listing_df.to_csv('analytical_database/dimension_listing.csv', index=False)

### Fact Listings

In [5]:
# Create Fact Listings
fact_listings_df = (
    listings_df.merge(
        dimension_neighborhood_df[['neighborhood_id', 'neighborhood_cleansed']],
        how='left',
        left_on='neighbourhood_cleansed',
        right_on='neighborhood_cleansed'
    )
    # Select only the desired columns after the merge
    [[
        'id', 'host_id', 'neighborhood_id', 'price', 'accommodates', 'bathrooms',
        'bedrooms', 'beds', 'review_scores_rating', 'review_scores_accuracy',
        'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
        'review_scores_location', 'review_scores_value', 'number_of_reviews', 
        'number_of_reviews_ltm', 'number_of_reviews_l30d', 'availability_30',
        'availability_60', 'availability_90', 'availability_365', 'reviews_per_month'
    ]]
    .rename(columns={'id': 'listing_id'})  # Rename 'id' to 'listing_id'
)

# Save the result to CSV
#fact_listings_df.to_csv('analytical_database/fact_listings.csv', index=False)

,listing_id,host_id,neighborhood_id,price,accommodates,bathrooms,bedrooms,beds,review_scores_rating,review_scores_accuracy,...,review_scores_location,review_scores_value,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_30,availability_60,availability_90,availability_365,reviews_per_month
0,31094,129976,1,NaN,6,1.5,4,0,4.88,4.82,...,4.8,4.53,19,0,0,0,0,0,0,0.11
1,31094,129976,4,NaN,6,1.5,4,0,4.88,4.82,...,4.8,4.53,19,0,0,0,0,0,0,0.11
2,31094,129976,5,NaN,6,1.5,4,0,4.88,4.82,...,4.8,4.53,19,0,0,0,0,0,0,0.11
3,31094,129976,8,NaN,6,1.5,4,0,4.88,4.82,...,4.8,4.53,19,0,0,0,0,0,0,0.11
4,31094,129976,16,NaN,6,1.5,4,0,4.88,4.82,...,4.8,4.53,19,0,0,0,0,0,0,0.11


In [6]:
fact_listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798836 entries, 0 to 798835
Data columns (total 23 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   listing_id                   798836 non-null  int64  
 1   host_id                      798836 non-null  int64  
 2   neighborhood_id              798836 non-null  int64  
 3   price                        518174 non-null  float64
 4   accommodates                 798836 non-null  int64  
 5   bathrooms                    798836 non-null  float64
 6   bedrooms                     798836 non-null  int64  
 7   beds                         798836 non-null  int64  
 8   review_scores_rating         680126 non-null  float64
 9   review_scores_accuracy       679261 non-null  float64
 10  review_scores_cleanliness    679261 non-null  float64
 11  review_scores_checkin        679261 non-null  float64
 12  review_scores_communication  679261 non-null  float64
 13 

### Dimension Host

In [6]:
# Create Dimension Host
dimension_host_df = listings_df[[
    'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
    'host_neighbourhood', 'host_about', 'host_response_time', 
    'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 
    'host_has_profile_pic', 'host_picture_url', 'host_identity_verified', 
    'host_verifications', 'calculated_host_listings_count',
    'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms'
]].drop_duplicates()

dimension_host_df.to_csv('analytical_database/dimension_host.csv', index=False)

### Fact Calendar

In [7]:
# Create Fact Calendar
fact_calendar_df = calendar_df[[
    'listing_id', 'date', 'available', 'price', 'minimum_nights', 'maximum_nights'
]].rename(columns={'price': 'calendar_price'})

fact_calendar_df.to_csv('analytical_database/fact_calendar.csv', index=False)

### Dimension Time

In [8]:
import pandas as pd
from datetime import datetime

# Generate date range
date_range = pd.date_range(start='2008-06-27', end='2025-07-01', freq='D')
dimension_time_df = pd.DataFrame({'date': date_range})

# Add columns
dimension_time_df['year'] = dimension_time_df['date'].dt.year
dimension_time_df['quarter'] = ((dimension_time_df['date'].dt.month - 1) // 3 + 1)
dimension_time_df['month'] = dimension_time_df['date'].dt.month
dimension_time_df['day'] = dimension_time_df['date'].dt.day
dimension_time_df['weekday'] = dimension_time_df['date'].dt.day_name()
dimension_time_df['is_weekend'] = dimension_time_df['weekday'].isin(['Saturday', 'Sunday'])

dimension_time_df.to_csv('analytical_database/dimension_time.csv', index=False)